In [2]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, timeframes, met_api, nve_api, numba_rl_utils
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict, Counter
import random
import sklearn
import statistics

In [3]:
# GLOBAL VARIABLES:
tf = timeframes.one_week
areas = ["NO5"]


In [7]:
L, M, H = utils.get_all_sets(timeframe= tf, areas = areas)
sup_market_names = ["FCR", "aFRR"]
markets = [market for market in M if sup_market_names[0] in market.name  or sup_market_names[1] in market.name]
#F, freq_data, _ = utils.get_frequency_sets(tf= tf, M =M, H= H)
#L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m = utils.get_parameters(L = L, M = M, H = H)
#Ir_hlm, Ia_hlm, Va_hm = utils.get_income_dictionaries(H=H, L = L, M = M, freq_data= freq_data, Fu_h_l= Fu_h_l, Fd_h_l= Fd_h_l, P_h_m= P_h_m, Vp_h_m= Vp_h_m, F = F, markets_dict = {market.name : market for market in M}, timeframe = tf, areas = areas)
#compatible_dict = utils.get_compatibility_dict(L = L ,M = M, index = False)

In [10]:
sup_market_names = ["FCR", "aFRR"]
markets = [market for market in M if sup_market_names[0] in market.name  or sup_market_names[1] in market.name]

In [9]:
up_volumes_hl, down_volumes_hl, market_prices_hm, market_volumes_hm, asset_directions_l, market_directions_m = numba_rl_utils.get_market_and_asset_values(H, L, M)


In [12]:
def get_expected_prices_and_volumes_dict(n_hours: int, market_volumes : np.array(float), market_prices : np.array(float), market_directions : np.array(int)):
    """ function to calculate the expected prices for each direction and area in the bid_timeframe

    Args:
        bid_timeframe (_type_): _description_
        markets (_type_): _description_

    Returns:
        _type_: _description_
    """
    expected_prices = np.zeros((3, n_hours)) # only NO5 is taken in to consideration
    expected_volumes = np.zeros((3, n_hours))
    for d in [-1, 0, 1]:
        #for area in ["NO1", "NO2", "NO3", "NO4", "NO5"]:
        for h in range(n_hours):
            expected_prices[(d, h)] = np.mean([market_prices[h, m] for m in range(len(market_directions)) if market_directions[m] == d])
            expected_volumes[(d, h)] = np.mean([market_volumes[h, m] for m in range(len(market_directions)) if market_directions[m] == d])
    return expected_prices, expected_volumes


In [13]:
exp_price_array_dh, exp_vol_array_dh = get_expected_prices_and_volumes_dict(n_hours = len(H), market_volumes = market_volumes_hm, market_prices = market_prices_hm, market_directions  = market_directions_m)


In [22]:
exp_price_array_dh.shape

(3, 168)

In [23]:
def normalize_array_vals(array, norm_method :str = "min_max" ):
    """Function to normalize the values of a dictionary

    Args:
        dict (dict): [description]
        norm_method (str): Normalization method; can choose between min-max normalization or z-score normalization. Defaults to "min_max".

    Returns:
        dict: the same dictionary as input, but with normalized values
    """
    if norm_method == "min_max":
        # check how mamy dimensions the array has
        if len(array.shape) >= 1:
            min_vals = array.min(axis=1, keepdims=True)
            max_vals = array.max(axis=1, keepdims=True)
            normalized_array = (array - min_vals) / (max_vals - min_vals)   
        else:
            # if the array has only one dimension, we can simply normalize the values
            min_value = np.min(array)
            max_value = np.max(array)
            # normalize the values in the array
            normalized_array = (array - min_value) / (max_value - min_value)
    else:
        if len(array.shape) >= 1:
            mean_vals = array.mean(axis=1, keepdims=True)
            std_vals = array.std(axis=1, keepdims=True)
            normalized_array = (array - mean_vals) / std_vals
        else:
            mean = statistics.mean(array)
            std = statistics.stdev(array)
            # Normalize and update the dictionary
            normalized_array = (array - mean) / std

    return normalized_array

In [24]:
norm_exp_price_array_dh = normalize_array_vals(exp_price_array_dh)
norm_exp_vol_array_dh = normalize_array_vals(exp_vol_array_dh)

In [26]:
compatible_array = numba_rl_utils.get_compatibility_array(L, M)

In [27]:
weather_dict = met_api.get_normalized_weather_dfs(reference_tf= timeframes.one_month, usage_tf = tf, areas = areas)
air_temp_array = weather_dict[("air_temperature", areas[0])].to_numpy()
wind_speed_array = weather_dict[("wind_speed", areas[0])].to_numpy()
precipitation_array = weather_dict[("sum(precipitation_amount P1D)", areas[0])].to_numpy()
cloud_cover_array = weather_dict[("cloud_area_fraction", areas[0])].to_numpy()
spot_path = "../master-data/spot_data/spot_june_23.csv"
norm_da_df = final_markets.preprocess_spot_data(pd.read_csv(spot_path), year = tf.year, start_month = tf.start_month, end_month = tf.end_month, start_day = tf.start_day, end_day = tf.end_day, start_hour = tf.start_hour, end_hour = tf.end_hour, area = areas[0], normalize= True)
norm_da_array_h = norm_da_df["settlement"].to_numpy()



In [ ]:
@numba.jit(nopython=True)
def train_nstep_model_numba_version(epsilon : float, alpha : float, gamma : float, num_episodes : int, \
                                    asset_indexes : np.array(int), market_indexes : np.array(int), \
                                    hour_indexes : np.array(int), norm_exp_price_array_dh : np.array(int),\
                                    norm_exp_vol_array_dh : np.array(int), air_temp_array : np.array(int), \
                                    wind_speed_array : np.array(int), precipitation_array : np.array(int), \
                                    cloud_cover_array : np.array(int), norm_da_array_h : np.array(int), \
                                    compatible_array : np.array(int), n_actions : int, n_features : int):
                                   
    """ Training function to learn how to bid in to the markets due to the bidding constraints and the asset constraints as well as optimizing the weights. 
    The function will learn how to bid in to the markets by updating the weights due to the features. 
    The change from v_4 is that this function will compare bids that are set to the same hour to get a better estimate of the reward.
    
    Args:
        epsilon (float): float number between 0 and 1, says how much the agent should explore
        alpha (float): float number between 0 and 1, also known as the learning rate
        gamma (float): float number between 0 and 1, also known as the discount factor
        num_episodes (int): number of episodes to be ran
        L ([new_meters.PowerMeter]): list of PowerMeter objects
        M ([final_markets.Reservemarket]): list of ReserveMarket objects
        H ([pd.Timestamp]): list of timestamps
        norm_exp_price_dict (dict): dictionary of the normalized expected prices for each direction and area in the bid_timeframe
        norm_exp_vol_dict (dict): dictionary of the normalized expected volumes for each direction and area in the bid_timeframe
        norm_w_df (pd.DataFrame) : normalized weather data in pd.dataframe format
        norm_da_df (pd.DataFrame) : normalized day ahead prices in pd.dataframe format
        n_actions (int): number of actions
        n_features (int): number of features

    Returns:
        bids (dict): dictionary that holds control over the final bids for each market and each hour. The keys are tuples of the market name and the hour and the values are the the assets that were bid for the given market and hour. The values are tuples including the list of assets and the aggregated volume.
        revenues (dict): dictionary that holds the revenue for each episode
        available_assets (dict): dictionary that holds the available assets for each hour
        episode_weights (dict): dictionary that holds the weights for each episode
        action_feature_dict: dictionary that holds the features for each action for each episode
        asset_bids (list): list that holds the bids for each episode. The indexes are the episode number and the values are dataframes that holds the bids for each market and each hour for each episode.
    """
    revenues_per_ep = np.zeros(num_episodes)
    bid_timeframe = hour_indexes[24:]
    #bids = {(market.name, hour): [] for hour in bid_timeframe for market in markets}
    "For each hour, an action should be to either bid in every feasible asset, to bid the minimum volume or to bid nothing"
    place_bid_hours = np.array([h for h in hour_indexes if h.hour in [7, 17, 18]])
    possible_hours, possible_markets_indexes = numba_rl_utils.get_possible_dates(place_bid_hours[0]) # the market_name is not the full name of the market, but rather a substring of the full name. Since FCR-D and FCR-N has the same deadlines there will be two markets with the same substring
    
    #indexes = [market_names.index(m.name) for m in possible_markets] # the indexes for each market. This is used to slice the Q-table so it is not necessary in this case
    features = numba_rl_utils.get_features(hour_index = place_bid_hours[0], available_assets = asset_indexes, 
                                           market_index = possible_markets_indexes[np.random.randint(0, 1)], expected_prices = norm_exp_price_array_dh, 
                                           expected_volumes =norm_exp_vol_array_dh, air_temp_array = air_temp_array, 
                                           precipitation_array = precipitation_array, wind_speed_array = wind_speed_array,
                                        cloud_cover_array = cloud_cover_array, da_array  = norm_da_array_h, market_directions_m = market_directions_m):

    
    # make a np.array which is equivalent to this dictionary available_assets = {hour: asset_indexes.copy() for hour in bid_timeframe} # 
    available_assets_array = np.array([asset_indexes for h in bid_timeframe]) # not sure if this will work

    action_0 = np.random.randint(0, n_actions-1)
    # will have to use a action_feature_array for each episode, market and hour where i can store the actions, new_actions, features and new_features
    action_feature_array = np.zeros((num_episodes, len(markets), len(bid_timeframe), 4), dtype = np.int32) # not sure if this will work
    weights = numba_rl_utils.initialize_weights(n_actions = n_actions, n_features = n_features)

    # extract hours from H where hour == 7, 17, 18
    epsilon_decay = epsilon/num_episodes
    alpha_decay = alpha/num_episodes
    episode_weights = {}
    for episode_n in range(num_episodes):
        if episode_n > episode_n/10:
            epsilon -= epsilon_decay
            alpha -= alpha_decay
        elif episode_n % 10 == 0:
            print(episode_n)
        revenue = 0 
        episode_bid_df = pd.DataFrame(columns= ["Market", "Hour", "Asset Count", "Total Flex Volume"])
        episode_bid_dict = {}
        reward_bid_hour_dict = {}
       
        #print(f" weights at the start of episode nr {episode_n}: {weights}")

        #available_assets = {hour: L.copy() for hour in bid_timeframe}
        available_assets = {hour: [l.meter_id for l in L] for hour in bid_timeframe}
        for place_hour in place_bid_hours: # the hours when the bids are placed from
            (possible_hours, market_name) = rl_utils.get_possible_dates(place_hour) # returns the possible hours for the market to place bids in
            if len(possible_hours) != 24:
                print(f"No bids for {place_hour}")
                print(f"possible_hours: {len(possible_hours)}")
                break
            #print(f"possible_hours when bidding: {possible_hours}")
            possible_markets = [m for m in markets if market_name in m.name] # because i am now using only no5 markets, this list should be of length 1 or 2.
            if possible_hours[0] not in bid_timeframe: # 
                break
            
            for current_market in possible_markets: # the markets that are possible to bid in
                #print(f"current_market : {current_market.name}")
                for bid_hour in possible_hours:
                    chosen_portfolio, reward, flex_vol = make_bid(current_market, bid_hour, action_0, available_assets[bid_hour], compatible_dict_meter_id_market, top_n = 600, iterations = 100, asset_volumes = asset_volumes) # get portfolio and reward for a bid - will have to figure out a better way to define reward
                    if bid_hour not in reward_bid_hour_dict.keys():
                        reward_bid_hour_dict[bid_hour] = [reward]
                    else:
                        reward_bid_hour_dict[bid_hour].append(reward)

                    available_assets[bid_hour] = [asset for asset in available_assets[bid_hour] if asset not in chosen_portfolio] # remove the assets that were bid from the available assets

                    if len(chosen_portfolio) > 0:
                        episode_bid_df.loc[len(episode_bid_df)] = [current_market.name, bid_hour, len(chosen_portfolio), flex_vol] # add the bid to the episode_bid_df

                    episode_bid_dict[(current_market.name, bid_hour)] = (chosen_portfolio, flex_vol) # add the bid to the bids dictionary
                    
                    new_features = get_features(bid_hour = bid_hour, available_assets= available_assets[bid_hour], 
                                                market = current_market, norm_da_df=norm_da_df, norm_exp_price_dict= norm_exp_price_dict, 
                                                norm_exp_vol_dict= norm_exp_vol_dict, precipitation_df= precipitation_df, cloud_cover_df= cloud_cover_df, 
                                                wind_speed_df= wind_speed_df, air_temp_df= air_temp_df, L = L, markets = markets) # update the features
                    #print(f"weights : {weights}")
                    possible_actions = [np.dot(weights[action], new_features) for action in range(n_actions)] 
                    #print(f"possible_actions: {possible_actions}")
                    new_action = rl_utils.greedy_action(possible_actions, epsilon)
                    action_feature_array[(episode_n, current_market.name, bid_hour)] = (action_0, new_action, features, new_features)
                    features, action_0 = new_features, new_action
                    revenue += reward
            #print([m.name for m in possible_markets])
            # Instead of updating the weights after each hour, the weights are updated after each market. This is to get a better estimate of the reward since the rewards are given after each of the markets are considererd for an hour.
            if "FCR_N_D_1_NO5" in [m.name for m in possible_markets]: # The FCR D-1 markets has the latest deadline, so after these markets has been considered, all markets has been considered for the given hour.
                #print(f"Possible hours when updating the weights : {possible_hours}")
                for bid_hour in possible_hours: 
                    total_reward = sum(reward_bid_hour_dict[bid_hour])
                    #print(f"total_reward: {total_reward}")
                    for market in markets:
                        if (episode_n, market.name, bid_hour) in action_feature_array.keys(): # should maybe rearrange the structure of the code so that this if statement is not necessary
                            action_0, new_action, features, new_features = action_feature_array[(episode_n, market.name, bid_hour)]
                            
                            q_values_next_state = np.array([np.dot(weights[a], new_features) for a in range(n_actions)])
                            
                            weights[action_0] = weights[action_0] + alpha * \
                            (total_reward/255 + gamma * np.max(q_values_next_state) - np.dot(weights[action_0], features)) * features
                            #print(f"weights after updating: {weights}")

        episode_weights[episode_n] = weights.copy()        
        bids[episode_n] = episode_bid_df.sort_values(by = ["Hour"])
        revenues[episode_n] = revenue
        asset_bids.append(episode_bid_dict)
    return bids, revenues, episode_weights, action_feature_array, asset_bids